In [ ]:
! pip install ollama 

In [ ]:
import ollama 

prompt = "Hello,ollama , give mahindra  car parts"
response = ollama.chat(model="qwen2.5vl:7b", messages=[{"role": "user", "content": prompt}])
print(response['message']['content'])

In [1]:
import ollama
import json
import re
from pdf2image import convert_from_path
from io import BytesIO
from pathlib import Path
import time


def pdf_to_images_bytes(pdf_path):
    """Convert all PDF pages to JPEG image bytes."""
    pages = convert_from_path(pdf_path, dpi=150)
    image_bytes_list = []
    for i, page in enumerate(pages):
        buf = BytesIO()
        page.save(buf, format="JPEG")
        image_bytes_list.append(buf.getvalue())
    return image_bytes_list


def extract_passport_from_pdf(pdf_path: str, model_name="qwen2.5vl:7b"):
    print(f"\n Processing PDF: {pdf_path}")
    start_time = time.time()

    image_bytes_list = pdf_to_images_bytes(pdf_path)

    prompt = """
    You are a senior OCR document understanding model.
    Analyze the uploaded passport or visa page image(s).

    Rules:
    - Keep both Arabic and English text exactly as visible.
    - Do NOT translate.
    - If a label (key) exists in both Arabic and English, combine them using " / ".
    - If a value exists in both Arabic and English, also combine them using " / ".
    - Output must be valid JSON only, no explanations.
    - Don't skip any visible fields — include all possible fields.

    Example format:
    {
      "ID Number / رقم الهوية": "784199787632597",
      "File No / رقم الملف": "201/2023/7/663922",
      "Passport No / رقم الجواز": "VT1337002",
      "Name / الاسم": "MUHAMMAD AMIR IQBAL MUHAMMAD I / محمد امير اقبال محمد اقبال",
      "Profession / المهنة": "PARTNER / شريك",
      "Employer / صاحب العمل": "H S P INTERNATIONAL FOODSTUFF TRADING L.L.C / اتش اس بي انترناشونال لتجارة المواد الغذائية ش ذ م م",
      "Place of Issue / جهة الإصدار": "دبي",
      "Issue Date / تاريخ إصدار الإقامة": "15/11/2023",
      "Expiry Date / تاريخ إنتهاء الإقامة": "14/11/2025",
      "Country / الدولة": "UNITED ARAB EMIRATES / دولة الإمارات العربية المتحدة",
      "Type / نوع الإقامة": "RESIDENCE / إقامة"
    }

    Only output JSON.
    """

    print(" Running model inference on all pages together...")
    response = ollama.chat(
        model=model_name,
        messages=[
            {
                "role": "user",
                "content": prompt,
                "images": image_bytes_list
            }
        ],
    )

    content = response["message"]["content"]

    # Try to safely parse JSON
    try:
        match = re.search(r"(\{[\s\S]*\})", content)
        data = json.loads(match.group(1)) if match else {"raw_text": content}
    except Exception as e:
        data = {"raw_text": content, "error": str(e)}

    output = {"file": str(pdf_path), "extracted_data": data}
    print(f"\n Extraction completed in {time.time() - start_time:.2f} seconds.")
    return output


In [3]:
# Example usage
pdf_path = Path(r"D:\AI Projects\passport_extraction\passport\2.In house PL 31.03.2025.pdf")  # <-- change to your file path
result = extract_passport_from_pdf(pdf_path)

# Display nicely formatted output
import json
print(json.dumps(result, indent=2, ensure_ascii=False))



 Processing PDF: D:\AI Projects\passport_extraction\passport\2.In house PL 31.03.2025.pdf
 Running model inference on all pages together...

 Extraction completed in 102.00 seconds.
{
  "file": "D:\\AI Projects\\passport_extraction\\passport\\2.In house PL 31.03.2025.pdf",
  "extracted_data": {
    "PARTICULARS / المخصصات": "TOTAL COMPREHENSIVE INCOME FOR THE YEAR / الدخل الإجمالي للسنة",
    "Amount [AED] / المبلغ [درهم إماراتي]": "1,541,978 / 1,541,978"
  }
}


below code stored data in json 

In [2]:
import ollama
import json
import re
from pdf2image import convert_from_path
from io import BytesIO
from pathlib import Path
import time


def pdf_to_images_bytes(pdf_path):
    """Convert all PDF pages to JPEG image bytes."""
    pages = convert_from_path(pdf_path, dpi=150)
    image_bytes_list = []
    for i, page in enumerate(pages):
        buf = BytesIO()
        page.save(buf, format="JPEG")
        image_bytes_list.append(buf.getvalue())
    return image_bytes_list


def extract_passport_from_pdf(pdf_path: str, model_name="qwen2.5vl:7b"):
    print(f"\nProcessing PDF: {pdf_path}")
    start_time = time.time()

    image_bytes_list = pdf_to_images_bytes(pdf_path)

    prompt = """
    You are a senior OCR document understanding model.
    Analyze the uploaded passport or visa page image(s).

    Rules:
    - Keep both Arabic and English text exactly as visible.
    - if no arabic words are present just keep english text as is.
    - Do NOT translate.
    - If a label (key) exists in both Arabic and English, combine them using " / ".
    - If a value exists in both Arabic and English, also combine them using " / ".
    - Output must be valid JSON only, no explanations.
    - Don't skip any visible fields — include all possible fields.

    Example format:
    {
      "ID Number / رقم الهوية": "784199787632597",
      "File No / رقم الملف": "201/2023/7/663922",
      "Passport No / رقم الجواز": "VT1337002",
      "Name / الاسم": "MUHAMMAD AMIR IQBAL MUHAMMAD I / محمد امير اقبال محمد اقبال",
      "Profession / المهنة": "PARTNER / شريك",
      "Employer / صاحب العمل": "H S P INTERNATIONAL FOODSTUFF TRADING L.L.C / اتش اس بي انترناشونال لتجارة المواد الغذائية ش ذ م م",
      "Place of Issue / جهة الإصدار": "دبي",
      "Issue Date / تاريخ إصدار الإقامة": "15/11/2023",
      "Expiry Date / تاريخ إنتهاء الإقامة": "14/11/2025",
      "Country / الدولة": "UNITED ARAB EMIRATES / دولة الإمارات العربية المتحدة",
      "Type / نوع الإقامة": "RESIDENCE / إقامة"
    }

    Only output JSON.
    """

    print("Running model inference on all pages together...")
    response = ollama.chat(
        model=model_name,
        messages=[
            {
                "role": "user",
                "content": prompt,
                "images": image_bytes_list
            }
        ],
    )

    content = response["message"]["content"]

    # Try to safely parse JSON
    try:
        match = re.search(r"(\{[\s\S]*\})", content)
        data = json.loads(match.group(1)) if match else {"raw_text": content}
    except Exception as e:
        data = {"raw_text": content, "error": str(e)}

    output = {"file": str(pdf_path), "extracted_data": data}

    # Save to a separate JSON file
    output_file = Path(pdf_path).with_suffix(".json")
    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(output, f, ensure_ascii=False, indent=2)

    print(f"\n Extraction completed in {time.time() - start_time:.2f} seconds.")
    print(f" Output saved to: {output_file}")
    return output


# Example usage
pdf_path = Path(r"D:\AI Projects\passport_extraction\passport\2.In house PL 31.03.2025.pdf")
result = extract_passport_from_pdf(pdf_path)

# Optional: print the JSON content on screen
print(json.dumps(result, indent=2, ensure_ascii=False))



Processing PDF: D:\AI Projects\passport_extraction\passport\2.In house PL 31.03.2025.pdf
Running model inference on all pages together...

 Extraction completed in 37.70 seconds.
 Output saved to: D:\AI Projects\passport_extraction\passport\2.In house PL 31.03.2025.json
{
  "file": "D:\\AI Projects\\passport_extraction\\passport\\2.In house PL 31.03.2025.pdf",
  "extracted_data": {
    "PARTICULARS / المخصصات": "TOTAL COMPREHENSIVE INCOME FOR THE YEAR / الدخل الإجمالي للسنة",
    "Amount [AED] / المبلغ [درهم إماراتي]": "1,541,978"
  }
}


below code is checking for getting perfect output of json 

In [ ]:
import ollama
import json
import re
from pdf2image import convert_from_path
from io import BytesIO
from pathlib import Path
import time


def pdf_to_images_bytes(pdf_path):
    """Convert all PDF pages to JPEG image bytes."""
    pages = convert_from_path(pdf_path, dpi=150)
    image_bytes_list = []
    for i, page in enumerate(pages):
        buf = BytesIO()
        page.save(buf, format="JPEG")
        image_bytes_list.append(buf.getvalue())
    return image_bytes_list


def extract_passport_from_pdf(pdf_path: str, model_name="qwen2.5vl:7b"):
    print(f"\nProcessing PDF: {pdf_path}")
    start_time = time.time()

    image_bytes_list = pdf_to_images_bytes(pdf_path)

    prompt = """
    You are a senior OCR document understanding model.
    Analyze the uploaded passport or visa page image(s).

    Rules:
    - Keep both Arabic and English text exactly as visible.
    - if no arabic words are present just keep english text as is.
    - Do NOT translate.
    - If a label (key) exists in both Arabic and English, combine them using " / ".
    - If a value exists in both Arabic and English, also combine them using " / ".
    - Output must be valid JSON only, no explanations.
    - Don't skip any visible fields — include all possible fields.

    Example format:
    {
      "ID Number / رقم الهوية": "784199787632597",
      "File No / رقم الملف": "201/2023/7/663922",
      "Passport No / رقم الجواز": "VT1337002",
      "Name / الاسم": "MUHAMMAD AMIR IQBAL MUHAMMAD I / محمد امير اقبال محمد اقبال",
      "Profession / المهنة": "PARTNER / شريك",
      "Employer / صاحب العمل": "H S P INTERNATIONAL FOODSTUFF TRADING L.L.C / اتش اس بي انترناشونال لتجارة المواد الغذائية ش ذ م م",
      "Place of Issue / جهة الإصدار": "دبي",
      "Issue Date / تاريخ إصدار الإقامة": "15/11/2023",
      "Expiry Date / تاريخ إنتهاء الإقامة": "14/11/2025",
      "Country / الدولة": "UNITED ARAB EMIRATES / دولة الإمارات العربية المتحدة",
      "Type / نوع الإقامة": "RESIDENCE / إقامة"
    }

    Only output JSON.
    """

    print("Running model inference on all pages together...")
    response = ollama.chat(
        model=model_name,
        messages=[
            {
                "role": "user",
                "content": prompt,
                "images": image_bytes_list
            }
        ],
    )

    content = response["message"]["content"]

    # Try to safely parse JSON
    try:
        match = re.search(r"(\{[\s\S]*\})", content)
        data = json.loads(match.group(1)) if match else {"raw_text": content}
    except Exception as e:
        data = {"raw_text": content, "error": str(e)}

    output = {"file": str(pdf_path), "extracted_data": data}

    # Save to a separate JSON file
    output_file = Path(pdf_path).with_suffix(".json")
    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(output, f, ensure_ascii=False, indent=2)

    print(f"\n Extraction completed in {time.time() - start_time:.2f} seconds.")
    print(f" Output saved to: {output_file}")
    return output


# Example usage
pdf_path = Path(r"D:\AI Projects\passport_extraction\passport\2.In house PL 31.03.2025.pdf")
result = extract_passport_from_pdf(pdf_path)

# Optional: print the JSON content on screen
print(json.dumps(result, indent=2, ensure_ascii=False))


below code is python script for template making 

In [3]:
! pip install reportlab

  Using cached charset_normalizer-3.4.4-cp313-cp313-win_amd64.whl.metadata (38 kB)
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------------------------------------- 2.0/2.0 MB 19.2 MB/s eta 0:00:00
Using cached charset_normalizer-3.4.4-cp313-cp313-win_amd64.whl (107 kB)

   ---------------------------------------- 0/2 [charset-normalizer]
   ---------------------------------------- 0/2 [charset-normalizer]
   ---------------------------------------- 0/2 [charset-normalizer]
   -------------------- ------------------- 1/2 [reportlab]
   -------------------- ------------------- 1/2 [reportlab]
   -------------------- ------------------- 1/2 [reportlab]
   -------------------- ------------------- 1/2 [reportlab]
   -------------------- ------------------- 1/2 [reportlab]
   -------------------- ------------------- 1/2 [reportlab]
   -------------------- ------------------- 1/2 [reportlab]
   -------------------- ------------------- 1/2 [reportlab]
   -------


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from reportlab.lib.pagesizes import A4
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Table, TableStyle
from reportlab.lib import colors
import json
from pathlib import Path


In [2]:
# Load the previously extracted JSON file
json_path = Path(r"D:\AI Projects\passport_extraction\passport\VISA_61734560.json")

with open(json_path, "r", encoding="utf-8") as f:
    data = json.load(f)

extracted_data = data.get("extracted_data", {})
extracted_data


{'ID Number / رقم الهوية': '784199787632597',
 'File No / رقم الملف': '201/2023/7/663922',
 'Passport No / رقم الجواز': 'VT1337002',
 'Name / الاسم': 'MUHAMMAD AMIR IQBAL MUHAMMAD I / محمد امير اقبال محمد اقبال',
 'Profession / المهنة': 'PARTNER / شريك',
 'Employer / صاحب العمل': 'H S P INTERNATIONAL FOODSTUFF TRADING L.L.C / اتش اس بي انترناشونال لتجارة المواد الغذائية ش ذ م م',
 'Place of Issue / جهة الإصدار': 'دبي',
 'Issue Date / تاريخ إصدار الإقامة': '15/11/2023',
 'Expiry Date / تاريخ إنتهاء الإقامة': '14/11/2025',
 'Country / الدولة': 'UNITED ARAB EMIRATES / دولة الإمارات العربية المتحدة',
 'Type / نوع الإقامة': 'RESIDENCE / إقامة'}

In [ ]:
def create_passport_template(output_pdf_path, extracted_data):
    # Convert Path object to string if needed
    if isinstance(output_pdf_path, Path):
        output_pdf_path = str(output_pdf_path)

    doc = SimpleDocTemplate(output_pdf_path, pagesize=A4)
    elements = []

    styles = getSampleStyleSheet()
    title_style = styles['Heading1']
    normal_style = styles['Normal']

    # Title
    elements.append(Paragraph("Passport / Visa Extracted Details", title_style))
    elements.append(Spacer(1, 20))

    # Convert extracted JSON into table format
    data_list = [["Field", "Value"]]
    for key, value in extracted_data.items():
        # Ensure all values are strings (in case of None or numeric)
        data_list.append([str(key), str(value)])

    # Create table
    table = Table(data_list, colWidths=[200, 330])
    table.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), colors.darkblue),
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
        ('ALIGN', (0, 0), (-1, -1), 'LEFT'),
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
        ('FONTSIZE', (0, 0), (-1, 0), 12),
        ('BOTTOMPADDING', (0, 0), (-1, 0), 10),
        ('BACKGROUND', (0, 1), (-1, -1), colors.beige),
        ('GRID', (0, 0), (-1, -1), 0.25, colors.black),
    ]))

    elements.append(table)
    doc.build(elements)

    print(f" PDF template created successfully: {output_pdf_path}")


In [10]:
output_pdf = Path(r"filled_passport_template.pdf")
create_passport_template(output_pdf, extracted_data)


✅ PDF template created successfully: filled_passport_template.pdf


generate empty template 

In [11]:
from reportlab.lib.pagesizes import A4
from reportlab.pdfgen import canvas
from reportlab.lib.units import inch
from pathlib import Path

def create_empty_passport_template(output_pdf_path):
    c = canvas.Canvas(str(output_pdf_path), pagesize=A4)
    width, height = A4

    # Title
    c.setFont("Helvetica-Bold", 18)
    c.drawCentredString(width / 2, height - 60, "Passport / Visa Data Page Template")

    # Subtitle
    c.setFont("Helvetica", 12)
    c.drawCentredString(width / 2, height - 80, "(For Automated Data Filling System)")

    # Starting coordinates
    x_label = 60
    x_field = 250
    y_start = height - 130
    line_height = 40

    fields = [
        ("Passport No / رقم الجواز",),
        ("Name / الاسم",),
        ("Nationality / الجنسية",),
        ("Date of Birth / تاريخ الميلاد",),
        ("Place of Issue / جهة الإصدار",),
        ("Issue Date / تاريخ الإصدار",),
        ("Expiry Date / تاريخ الانتهاء",),
        ("Profession / المهنة",),
        ("Employer / صاحب العمل",),
        ("Country / الدولة",)
    ]

    c.setFont("Helvetica", 12)

    for i, (label,) in enumerate(fields):
        y = y_start - i * line_height
        c.drawString(x_label, y, f"{label} :")
        c.line(x_field, y - 2, width - 60, y - 2)  # Draw line for data

    # Border Box
    c.setLineWidth(2)
    c.rect(50, y_start - len(fields) * line_height - 20, width - 100, len(fields) * line_height + 50, stroke=1, fill=0)

    # Footer
    c.setFont("Helvetica-Oblique", 9)
    c.drawRightString(width - 60, 40, "Generated by Passport Extraction System")

    c.save()
    print(f"✅ Empty passport template created at: {output_pdf_path}")
       

In [12]:
output_path = Path(r"D:\AI Projects\passport_extraction\passport\passport_template_empty.pdf")
create_empty_passport_template(output_path)


✅ Empty passport template created at: D:\AI Projects\passport_extraction\passport\passport_template_empty.pdf


Use your extracted JSON + LLM to automatically map and fill values inside this template.

In [13]:
from reportlab.lib.pagesizes import A4
from reportlab.pdfgen import canvas
from PyPDF2 import PdfReader, PdfWriter
import ollama, json
from pathlib import Path


In [14]:
# Paths
json_path = Path(r"D:\AI Projects\passport_extraction\passport\2.In house PL 31.03.2025.json")
template_pdf = Path(r"D:\AI Projects\passport_extraction\passport\passport_template_empty.pdf")

with open(json_path, "r", encoding="utf-8") as f:
    extracted_data = json.load(f).get("extracted_data", {})

# ---- Optional: ask the LLM to normalize key names ----
prompt = f"""
You are a mapping assistant. Match the JSON keys to these template fields:

["Passport No", "Name", "Nationality", "Date of Birth", "Place of Issue",
 "Issue Date", "Expiry Date", "Profession", "Employer", "Country"]

Return JSON with this format:
{{
  "Passport No": "<value>",
  "Name": "<value>",
  ...
}}
JSON input:
{json.dumps(extracted_data, ensure_ascii=False, indent=2)}
"""

response = ollama.chat(model="qwen2.5:7b", messages=[{"role": "user", "content": prompt}])
mapped_data_text = response["message"]["content"]

try:
    mapped_data = json.loads(mapped_data_text)
except Exception:
    mapped_data = extracted_data  # fallback if parsing fails

mapped_data


{'PARTICULARS / المخصصات': 'TOTAL COMPREHENSIVE INCOME FOR THE YEAR / الدخل الإجمالي للسنة',
 'Amount [AED] / المبلغ [درهم إماراتي]': '1,541,978'}

In [15]:
def fill_passport_template(empty_pdf_path, output_pdf_path, data):
    reader = PdfReader(str(empty_pdf_path))
    writer = PdfWriter()

    # Get template page
    page = reader.pages[0]
    packet_path = output_pdf_path.with_suffix(".overlay.pdf")

    width, height = A4
    c = canvas.Canvas(str(packet_path), pagesize=A4)
    c.setFont("Helvetica", 12)

    # Coordinates must match the template layout (same y positions)
    x_field = 250
    y_start = height - 130
    line_height = 40

    fields_order = [
        "Passport No", "Name", "Nationality", "Date of Birth",
        "Place of Issue", "Issue Date", "Expiry Date",
        "Profession", "Employer", "Country"
    ]

    for i, field in enumerate(fields_order):
        y = y_start - i * line_height
        value = data.get(field, "")
        c.drawString(x_field + 5, y - 10, str(value))

    c.save()

    # Merge overlay with template
    overlay_reader = PdfReader(str(packet_path))
    overlay_page = overlay_reader.pages[0]
    page.merge_page(overlay_page)
    writer.add_page(page)

    with open(output_pdf_path, "wb") as f:
        writer.write(f)

    Path(packet_path).unlink(missing_ok=True)
    print(f"✅ Filled passport saved at: {output_pdf_path}")


In [ ]:
filled_pdf = Path(r"D:\AI Projects\passport_extraction\passport\passport_filled.pdf")
fill_passport_template(template_pdf, filled_pdf, mapped_data)


fill form

In [7]:

! pip install ollama pdf2image pymupdf PyPDF2


  Using cached pypdf2-3.0.1-py3-none-any.whl.metadata (6.8 kB)
   ---------------------------------------- 0.0/18.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/18.7 MB ? eta -:--:--
    --------------------------------------- 0.3/18.7 MB ? eta -:--:--
   --------- ------------------------------ 4.5/18.7 MB 17.1 MB/s eta 0:00:01
   ------------------------- -------------- 11.8/18.7 MB 25.9 MB/s eta 0:00:01
   ------------------------------------ --- 17.0/18.7 MB 25.8 MB/s eta 0:00:01
   ---------------------------------------- 18.7/18.7 MB 23.6 MB/s eta 0:00:00
Using cached pypdf2-3.0.1-py3-none-any.whl (232 kB)

   ---------------------------------------- 0/2 [PyPDF2]
   ---------------------------------------- 0/2 [PyPDF2]
   ---------------------------------------- 0/2 [PyPDF2]
   ---------------------------------------- 0/2 [PyPDF2]
   ---------------------------------------- 0/2 [PyPDF2]
   ---------------------------------------- 0/2 [PyPDF2]
   --------------


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


fill form

In [ ]:
import json
from pathlib import Path
from pdf2image import convert_from_path
from io import BytesIO
import re
import ollama

# === File Paths ===
empty_form_pdf = Path(r"D:\AI Projects\passport_extraction\passport\23.pdf")
json_file = Path(r"D:\AI Projects\passport_extraction\passport\VISA_61734560.json")

# Load extracted JSON data
with open(json_file, "r", encoding="utf-8") as f:
    data = json.load(f)
    extracted_data = data.get("extracted_data", data)

print(" Loaded Extracted JSON Keys:")
for k in extracted_data.keys():
    print("-", k)

# === Convert ALL Pages of the Empty PDF to Images ===
pages = convert_from_path(str(empty_form_pdf), dpi=150)
page_images = []
for p in pages:
    buf = BytesIO()
    p.save(buf, format="JPEG")
    page_images.append(buf.getvalue())

print(f"\n📄 Converted {len(page_images)} pages to images for LLM analysis.\n")

✅ Loaded Extracted JSON Keys:
- ID Number / رقم الهوية
- File No / رقم الملف
- Passport No / رقم الجواز
- Name / الاسم
- Profession / المهنة
- Employer / صاحب العمل
- Place of Issue / جهة الإصدار
- Issue Date / تاريخ إصدار الإقامة
- Expiry Date / تاريخ إنتهاء الإقامة
- Country / الدولة
- Type / نوع الإقامة

📄 Converted 7 pages to images for LLM analysis.



In [34]:
# === Analyze Each Page with Qwen2.5-VL ===
all_fields = {}

for i, img_bytes in enumerate(page_images):
    print(f"🔍 Processing Page {i+1}...")
    
    prompt = f"""
    You are an expert in reading document layouts.
    The uploaded image is page {i+1} of an EMPTY Indian passport form.

    Task:
    - Identify every field label (like File Number, Name, Country, Profession, etc.).
    - Output their coordinates in pixels relative to the top-left corner.
    - Keep both Arabic and English text exactly as visible.
    - Output must be JSON only.

    Format:
    {{
      "fields": {{
        "File Number (For Office Use Only)": {{ "x": 220, "y": 180 }},
        "Name / الاسم": {{ "x": 250, "y": 420 }}
      }}
    }}

    Image size is 150 DPI.
    """

    response = ollama.chat(
        model="qwen2.5vl:7b",
        messages=[{"role": "user", "content": prompt, "images": [img_bytes]}],
    )

    content = response["message"]["content"]
    match = re.search(r"(\{[\s\S]*\})", content)
    fields_info = json.loads(match.group(1)) if match else {}

    all_fields[f"page_{i+1}"] = fields_info.get("fields", {})

print("\n✅ Finished detecting all field coordinates from all pages!\n")
print(json.dumps(all_fields, indent=2, ensure_ascii=False))

🔍 Processing Page 1...
🔍 Processing Page 2...
🔍 Processing Page 3...
🔍 Processing Page 4...
🔍 Processing Page 5...
🔍 Processing Page 6...
🔍 Processing Page 7...

✅ Finished detecting all field coordinates from all pages!

{
  "page_1": {
    "File Number (For Office Use Only)": {
      "x": 220,
      "y": 180
    },
    "Name / الاسم": {
      "x": 250,
      "y": 420
    }
  },
  "page_2": {
    "2.4 Date of Birth (DD-MM-YYYY)": {
      "x": 220,
      "y": 180
    },
    "2.5 Place of Birth (Village or Town or City)": {
      "x": 250,
      "y": 420
    },
    "2.6 Gender": {
      "x": 220,
      "y": 660
    },
    "2.7 Marital Status": {
      "x": 220,
      "y": 720
    },
    "2.8 Citizenship of India": {
      "x": 220,
      "y": 780
    },
    "2.9 PAN (If available)": {
      "x": 220,
      "y": 840
    },
    "2.10 Voter ID (If available)": {
      "x": 220,
      "y": 900
    },
    "2.11 Employment Type": {
      "x": 220,
      "y": 1040
    },
    "2.12 If employed 

In [36]:
import json
import re
import ollama

# === Get all PDF field names (from multi-page detection) ===
# If you ran the multi-page detection earlier, use `all_fields`
# Otherwise, for single page, you can still use `fields_info`
all_pdf_fields = []

if "all_fields" in locals():
    # Multi-page form: collect all unique field names from all pages
    for page_num, fields_dict in all_fields.items():
        all_pdf_fields.extend(list(fields_dict.keys()))
else:
    # Single page fallback
    all_pdf_fields = list(fields_info.get("fields", {}).keys())

print(f" Total PDF fields detected: {len(all_pdf_fields)}")

# === Prepare Matching Prompt ===
prompt = f"""
You are a form-field mapping assistant.

I have two lists:
1️⃣ JSON keys (from extracted passport or visa data)
2️⃣ PDF field labels (from the empty passport form)

Your job:
- Match each JSON key to the most appropriate PDF field label, even if the wording differs.
- Be flexible with synonyms (e.g., "File No" ≈ "File Number", "Name" ≈ "Applicant's Given Name").
- Output valid JSON only — no explanations.

Example:
{{
  "File No / رقم الملف": "File Number (For Office Use Only)",
  "Name / الاسم": "Applicant's Given Name",
  "Profession / المهنة": "Profession",
  "Country / الدولة": "Country / Region"
}}

JSON keys: value 
{list(extracted_data.keys())}

PDF fields:
{all_pdf_fields}
"""

# === Use a Text Model for Better Matching (no image needed) ===
response = ollama.chat(
    model="qwen2.5vl:7b",
    messages=[{"role": "user", "content": prompt}],
)

# === Extract JSON Safely from Response ===
content = response["message"]["content"]
match = re.search(r"(\{[\s\S]*\})", content)
field_map = json.loads(match.group(1)) if match else {}

print("\n LLM Field Mapping:")
print(json.dumps(field_map, indent=2, ensure_ascii=False))


📄 Total PDF fields detected: 79

✅ LLM Field Mapping:
{
  "ID Number / رقم الهوية": "2.17 Aadhaar Number",
  "File No / رقم الملف": "File Number (For Office Use Only)",
  "Passport No / رقم الجواز": "2.12 If employed in Government/Statutory Body/PSU, specify organization name",
  "Name / الاسم": "Name / الاسم",
  "Profession / المهنة": "2.11 Employment Type",
  "Employer / صاحب العمل": "2.12 If employed in Government/Statutory Body/PSU, specify organization name",
  "Place of Issue / جهة الإصدار": "Name of passport office where applied",
  "Issue Date / تاريخ إصدار الإقامة": "Date (DD-MM-YYYY)",
  "Expiry Date / تاريخ إنتهاء الإقامة": "Date (DD-MM-YYYY)",
  "Country / الدولة": "Country / Region",
  "Type / نوع الإقامة": "2.15 Are you eligible for Non-ECR category?"
}


In [ ]:
# below code for convert empty pdf into html format

In [45]:
import fitz  # PyMuPDF

pdf_path = r"D:\AI Projects\passport_extraction\passport\23.pdf"
html_output = ""

# Open the PDF
doc = fitz.open(pdf_path)

# Extract all pages as HTML
for page_num, page in enumerate(doc, start=1):
    html_output += f"<!-- Page {page_num} -->\n"
    html_output += page.get_text("html")

# Save HTML to file
html_path = r"output_from_pymupdf.html"
with open(html_path, "w", encoding="utf-8") as f:
    f.write(html_output)

print(f"✅ Saved HTML file: {html_path}")


✅ Saved HTML file: output_from_pymupdf.html


In [18]:
import ollama, json, re
from pdf2image import convert_from_path
from io import BytesIO

def pdf_to_images_bytes(pdf_path):
    pages = convert_from_path(pdf_path, dpi=150)
    img_bytes = []
    for page in pages:
        buf = BytesIO()
        page.save(buf, format="JPEG")
        img_bytes.append(buf.getvalue())
    return img_bytes


def extract_and_map_passport(pdf_path, model="qwen2.5vl:7b"):
    print("📄 Extracting and mapping passport data...")

    schema = {
        "PassportNumber": "",
        "Surname": "",
        "GivenNames": "",
        "DateOfBirth": "",
        "PlaceOfBirth": "",
        "Nationality": "",
        "Sex": "",
        "DateOfIssue": "",
        "DateOfExpiry": "",
        "Authority": ""
    }

    prompt = f"""
    You are an expert OCR + document understanding LLM.
    Extract ALL data from the given passport image(s).
    Then map each value to the following schema strictly:
    {json.dumps(schema, indent=2)}

    Rules:
    - Keep Arabic and English text as seen (no translation).
    - If a value appears in both Arabic and English, join with " / ".
    - If a field is not visible, keep it empty ("").
    - Output only valid JSON matching the schema.
    """

    images = pdf_to_images_bytes(pdf_path)
    response = ollama.chat(
        model=model,
        messages=[{"role": "user", "content": prompt, "images": images}]
    )

    text = response["message"]["content"]
    match = re.search(r"\{[\s\S]*\}", text)
    return json.loads(match.group()) if match else {}


In [19]:
from reportlab.lib.pagesizes import A4
from reportlab.lib import colors
from reportlab.pdfgen import canvas
from pathlib import Path

def create_passport_template(path="passport_template.pdf"):
    c = canvas.Canvas(path, pagesize=A4)
    width, height = A4

    c.setTitle("Passport Template")

    c.setFont("Helvetica-Bold", 14)
    c.drawCentredString(width / 2, height - 50, "Passport Data Page")

    c.setFont("Helvetica", 10)
    y = height - 100
    fields = [
        "PassportNumber", "Surname", "GivenNames", "DateOfBirth",
        "PlaceOfBirth", "Nationality", "Sex", "DateOfIssue",
        "DateOfExpiry", "Authority"
    ]

    for f in fields:
        c.drawString(80, y, f + ":")
        c.line(200, y, 500, y)
        y -= 30

    c.save()
    print(f"✅ Template created: {path}")


In [20]:
from PyPDF2 import PdfReader, PdfWriter
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import A4
from io import BytesIO

def fill_passport_template(json_data, template_pdf, output_pdf):
    packet = BytesIO()
    can = canvas.Canvas(packet, pagesize=A4)
    can.setFont("Helvetica", 10)

    y = A4[1] - 100
    for key, value in json_data.items():
        can.drawString(200, y, value)
        y -= 30

    can.save()
    packet.seek(0)

    new_pdf = PdfReader(packet)
    existing_pdf = PdfReader(template_pdf)
    output = PdfWriter()

    page = existing_pdf.pages[0]
    page.merge_page(new_pdf.pages[0])
    output.add_page(page)

    with open(output_pdf, "wb") as f:
        output.write(f)

    print(f"✅ Filled PDF created: {output_pdf}")


In [23]:
from pathlib import Path

pdf_path = Path(r"D:\AI Projects\passport_extraction\passport\2.In house PL 31.03.2025.pdf")
template_path = Path(r"D:\AI Projects\passport_extraction\passport\passport_template_empty.pdf")
output_path = Path(r"D:\AI Projects\passport_extraction\passport\filled_passport.pdf")

# Step 1: LLM extract + map
data = extract_and_map_passport(pdf_path)
print(json.dumps(data, indent=2, ensure_ascii=False))

# Step 2: Fill template
fill_passport_template(data, template_path, output_path)


📄 Extracting and mapping passport data...
{
  "PassportNumber": "",
  "Surname": "",
  "GivenNames": "",
  "DateOfBirth": "",
  "PlaceOfBirth": "",
  "Nationality": "",
  "Sex": "",
  "DateOfIssue": "",
  "DateOfExpiry": "",
  "Authority": ""
}
✅ Filled PDF created: D:\AI Projects\passport_extraction\passport\filled_passport.pdf


In [ ]:
from docx import Document
from docx.shared import Pt, RGBColor, Inches
from docx.enum.text import WD_ALIGN_PARAGRAPH

def create_stylish_bank_template(path: str = "stylish_bank_template.docx"):
    doc = Document()

    # --- Title ---
    title = doc.add_paragraph("Federal Bank - Account Opening Form")
    title.alignment = WD_ALIGN_PARAGRAPH.CENTER
    title.runs[0].font.size = Pt(18)
    title.runs[0].font.bold = True
    title.runs[0].font.color.rgb = RGBColor(0, 51, 153)  # Navy blue

    doc.add_paragraph("")  # blank line for spacing

    # --- Subtitle ---
    subtitle = doc.add_paragraph("Please provide your personal details accurately. All fields are mandatory.")
    subtitle.alignment = WD_ALIGN_PARAGRAPH.CENTER
    subtitle.runs[0].font.size = Pt(10)
    subtitle.runs[0].font.color.rgb = RGBColor(80, 80, 80)

    doc.add_paragraph("")  # blank line for spacing

    # --- Add Table for Information ---
    table = doc.add_table(rows=5, cols=2)
    table.style = "Table Grid"
    table.autofit = True

    fields = [
        ("Full Name", "{{name}}"),
        ("Age", "{{age}}"),
        ("Address", "{{address}}"),
        ("Account Number", "{{account_number}}"),
        ("IFSC Code", "{{ifsc_code}}")
    ]

    for i, (label, placeholder) in enumerate(fields):
        cell_label = table.cell(i, 0)
        cell_label.text = label
        for run in cell_label.paragraphs[0].runs:
            run.font.bold = True
            run.font.size = Pt(11)
            run.font.color.rgb = RGBColor(0, 0, 0)
        cell_value = table.cell(i, 1)
        cell_value.text = placeholder
        for run in cell_value.paragraphs[0].runs:
            run.font.size = Pt(11)
            run.font.color.rgb = RGBColor(50, 50, 50)

    doc.add_paragraph("")  # spacing

    # --- Declaration ---
    declaration = doc.add_paragraph(
        "I hereby declare that the above information provided is true and correct to the best of my knowledge."
    )
    declaration.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY
    declaration.runs[0].font.size = Pt(11)

    doc.add_paragraph("")  # spacing

    # --- Signature Section ---
    signature_table = doc.add_table(rows=1, cols=2)
    signature_table.style = "Table Grid"
    signature_table.autofit = True
    signature_table.cell(0, 0).text = "Signature of Applicant: ___________________________"
    signature_table.cell(0, 1).text = "Date: ___________________"

    # --- Footer ---
    doc.add_paragraph("")  # blank line
    footer = doc.add_paragraph("Federal Bank Ltd © 2025 | Confidential Document")
    footer.alignment = WD_ALIGN_PARAGRAPH.CENTER
    for run in footer.runs:
        run.font.size = Pt(9)
        run.font.color.rgb = RGBColor(120, 120, 120)

    doc.save(path)
    print(f"[💎] Stylish bank form template created successfully → {path}")

create_stylish_bank_template()
 